In [1]:
import tensorflow as tf
from tensorflow.python.framework import ops
import tflearn
import poisson_extensions
poisson_extensions.register_poisson_gradient(tf, ops)

In [2]:
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation

In [3]:
# Data loading and preprocessing
from tflearn.datasets import cifar10
(X, Y), (X_test, Y_test) = cifar10.load_data()
X, Y = shuffle(X, Y)
Y = to_categorical(Y, 10)
Y_test = to_categorical(Y_test, 10)

In [4]:
# Real-time data preprocessing
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()

# Real-time data augmentation
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)

In [5]:
# Convolutional network building, this dropout model achieves .8 accuracy, same for poisson used in final 2 layers
inputs = input_data(shape=[None, 32, 32, 3],
                     data_preprocessing=img_prep,
                     data_augmentation=img_aug)
conv1 = conv_2d(inputs, 32, 3, activation='leaky_relu')
conv1 = conv_2d(inputs, 32, 3, activation='leaky_relu')
pool1 = max_pool_2d(conv1, 2)
spike1 = dropout(pool1, .8)
# spike1 = tf.random_poisson(pool1, [])
after1 = tf.identity(spike1)
conv2 = conv_2d(after1, 64, 3, activation='leaky_relu')
conv2 = conv_2d(conv2, 64, 3, activation='leaky_relu')
pool2 = max_pool_2d(conv2, 2)
spike2 = dropout(pool2, .7)
# spike2 = tf.random_poisson(pool2, [])
after2 = tf.identity(spike2)
conv3 = conv_2d(after2, 128, 3, activation='leaky_relu')
conv3 = conv_2d(conv3, 128, 3, activation='leaky_relu')
pool3 = max_pool_2d(conv3, 2)
spike3 = dropout(pool3, 0.6)
# spike3 = tf.random_poisson(pool3, [])
after3 = tf.identity(spike3)
fc1 = fully_connected(after3, 512, activation='relu')
spike4 = dropout(fc1, 0.5)
# spike4 = tf.random_poisson(fc1, [])
after4 = tf.identity(spike4)
fc2 = fully_connected(after4, 10, activation='softmax')
net = regression(fc2, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)

In [6]:
model = tflearn.DNN(net, tensorboard_verbose=0)

In [ ]:
model.fit(X, Y, n_epoch=26, shuffle=True, validation_set=(X_test, Y_test),
          show_metric=True, batch_size=96, run_id='cifar10_cnn')

Training Step: 2549  | total loss: 0.96314 | time: 16.397s
| Adam | epoch: 005 | loss: 0.96314 - acc: 0.6660 -- iter: 44640/50000


In [20]:
model.evaluate(X_test, Y_test)

[0.81330000000000002]